In [19]:
from helpers import *
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)

In [20]:
s3=s3_connection()

connection successful


### La motivation

In [21]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
print(regional_stat[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count  36632.00    34720.00  35222.00   35065.00
mean    1829.92    17049.74      0.27       0.12
std     9954.65     5194.78      0.17       0.13
min        0.00     3350.48      0.00       0.00
25%      182.00    14011.97      0.16       0.00
50%      428.00    16321.86      0.26       0.08
75%     1116.00    19124.01      0.37       0.18
max   875791.00   126903.91      1.00       1.00


In [22]:
df=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
print(df[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count   2808.00     2808.00   2808.00    2808.00
mean   18078.54    18221.52      0.29       0.14
std    50290.90     6608.60      0.13       0.11
min       55.00     5818.04      0.00       0.00
25%     1570.25    14922.08      0.20       0.07
50%     4067.50    16931.46      0.28       0.12
75%    12203.00    19639.30      0.37       0.19
max   875791.00   107572.81      1.00       0.82


On voit que la population moyenne diminue lorsque l'on sélectionne les communes ayant une gare. Cela correspond au fait que les petites communes ont moins de chance d'avoir une gare. On passe de 36000 communes à 2800 communes, ce qui donne tout de même beaucoup d'observations. On remarque que le revenu moyen d'une commune en France est plus élevé lorsque l'on sélectionne seulement les communes comprenant une gare. De même, les personnes sont en moyenne plus diplômée et appartiennent aux catégories socio-professionnelles plus élevées également. 

On veut montrer qu'il existe un lien entre le revenu, la catégorie socio-professionnelle et le fait de prendre un train en France. Pour cela, on effectue une régression linéaire avec le nombre total de voyageurs comme variable dépendante. Notre but est d'expliquer pourquoi est-ce que cette variable est différente en fonction des communes, en contrôlant par la population de la commune. Les premières visualisations et les premières données suggèrent qu'il peut y avoir un lien entre son revenu et le fait de prendre un train.

### Création d'une nouvelle variable

In [23]:
df['freq_par_hab']= df['total_voyageurs_2022']/df['pop2022']
print(df[["freq_par_hab"]].head())

   freq_par_hab
0         18.09
1          4.52
2         90.82
3          0.88
4          1.40


In [24]:
print(df.columns)

Index(['total_voyageurs_2022', 'nom_gare', 'uic', 'codeinsee', 'lon_gare',
       'lat_gare', 'codecommune', 'nomcommune', 'pop2022', 'revmoy2022',
       'pbac2022', 'psup2022', 'pagri2022', 'pindp2022', 'pcadr2022',
       'pempl2022', 'pouvr2022', 'pchom2022', 'dist_closest_station_km',
       'nb_stations_same_commune', 'regions', 'freq_par_hab'],
      dtype='object')


In [8]:
communes_plus_1_gare = df[df['nb_stations_same_commune'] > 1]
# Nombre total de communes avec plus d'une gare
nombre_communes = len(communes_plus_1_gare)

# Afficher les résultats
print(f"Nombre de communes avec plus d'une gare : {nombre_communes}")
print(communes_plus_1_gare)

Nombre de communes avec plus d'une gare : 522
      total_voyageurs_2022                      nom_gare       uic codeinsee  \
3                    32800                          Agay  87757559     83118   
14                  586872                       Andrésy  87381491     78015   
18                16033835                    Argenteuil  87381848     95018   
19                   16320                    Argentière  87746859     74056   
32                   53740                     Babinière  87590349     44035   
...                    ...                           ...       ...       ...   
2781                 97742           Villaines-sous-Bois  87272021     95056   
2789               3855727  Villeparisis - Mitry-le-Neuf  87271510     77294   
2792                 53041           Villiers Montbarbin  87116749     77142   
2797               3770683               Vitry-sur-Seine  87545293     94081   
2802                 69296                        Wavrin  87286419     596

### Régressions

In [14]:
df['pnodip2022']=1-df['pbac2022']
df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]=df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]*100

KeyError: "['ppint2022'] not in index"

In [ ]:

X = df[['revmoy2022', 'pop2022', 'pnodip2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022','ppint2022', 'pagri2022', 'dist_closest_station_km']]
Y = df['total_voyageurs_2022']  

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

KeyError: "['ppint2022'] not in index"

In [ ]:
df['log_nb_voy2022']= np.log(df['total_voyageurs_2022'])

In [12]:
X = df[['revmoy2022', 'pop2022', 'pnodip2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022', 'dist_closest_station_km']]
Y = df['log_nb_voy2022']  

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         log_nb_voy2022   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     63.38
Date:                Tue, 17 Dec 2024   Prob (F-statistic):          3.31e-106
Time:                        09:31:14   Log-Likelihood:                -6297.1
No. Observations:                2808   AIC:                         1.261e+04
Df Residuals:                    2798   BIC:                         1.267e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     